In [1]:
import os
import dill
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False

import sys
sys.path.insert(0, '../../../src')

from biae import BayesianIQAEResult

In [2]:
# First, read all the data files. It takes about 5 minutes
thetas = range(0, 91, 1)  
all_data = {}

for theta in thetas:
    file_path = os.path.join('../all_angle_results', f'results_{theta}.pkl')
    with open(file_path, 'rb') as f:
        data_loaded = dill.load(f)
        # Extract only the needed data
        p = data_loaded['p']
        R = len(data_loaded['results_bayes'][0]['results'])
        all_data[theta] = {
            'bayes': [],
            'jeff': []
        }

        for method in ['results_bayes', 'results_jeff']:
            for i in range(len(data_loaded[method])):   # iterate over epsilons
                epsilon_target = data_loaded[method][i]['epsilon']
                num_correct = 0
                N_oracles = []
                for j in range(R):
                    iae_result = data_loaded[method][i]['results'][j]
                    ci = iae_result.confidence_interval
                    if ci[0] <= p and ci[1] >= p and (ci[1] - ci[0]) < 2 * epsilon_target:
                        num_correct += 1
                    N_oracles.append(iae_result.num_oracle_queries)
                proportion_correct = num_correct / R
                N_oracles_array = np.array(N_oracles)
                result = {
                    'epsilon': epsilon_target,
                    'proportion_correct': proportion_correct,
                    'average_N_oracles': np.mean(N_oracles_array),
                    'std_N_oracles': np.std(N_oracles_array)
                }
                if method == 'results_bayes':
                    all_data[theta]['bayes'].append(result)
                else:
                    all_data[theta]['jeff'].append(result)

        # Clear the loaded data to free memory
        del data_loaded

In [3]:
epsilons = [res['epsilon'] for res in all_data[thetas[0]]['bayes'][1:]]
epsilons

[0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07, 1e-08]

In [4]:
# Create a plot for each epsilon
for eps_idx, epsilon in enumerate(epsilons):
    # Create figure with controlled size
    fig, ax = plt.subplots(figsize=(10, 7))  # Fixed figure size
    
    # Extract data for this epsilon
    bayes_data = []
    jeff_data = []
    x_data = []  # For sin^2(theta)
    
    for theta in thetas:
        # Calculate sin^2(theta)
        x = np.sin(np.radians(theta))**2
        x_data.append(x)
        
        # Find the entry with matching epsilon
        bayes_entry = next(res for res in all_data[theta]['bayes'] if res['epsilon'] == epsilon)
        jeff_entry = next(res for res in all_data[theta]['jeff'] if res['epsilon'] == epsilon)
        
        bayes_data.append(bayes_entry['average_N_oracles'])
        jeff_data.append(jeff_entry['average_N_oracles'])
    
    # Fit beta*sqrt(a(1-a)) for both methods
    def fit_func(x, beta):
        return beta * np.sqrt(x * (1-x))
    
    from scipy.optimize import curve_fit
    
    # Calculate R² for fits
    def calculate_r2(y_true, y_pred):
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        return 1 - (ss_res / ss_tot)
    
    # Fit for Bayes data
    popt_bayes, _ = curve_fit(fit_func, x_data, bayes_data)
    beta_bayes = popt_bayes[0]
    y_pred_bayes = fit_func(np.array(x_data), beta_bayes)
    r2_bayes = calculate_r2(bayes_data, y_pred_bayes)
    
    # Fit for Jeff data
    popt_jeff, _ = curve_fit(fit_func, x_data, jeff_data)
    beta_jeff = popt_jeff[0]
    y_pred_jeff = fit_func(np.array(x_data), beta_jeff)
    r2_jeff = calculate_r2(jeff_data, y_pred_jeff)
    
    # Create smooth curve for plotting
    x_smooth = np.linspace(0, 1, 200)
    y_fit_bayes = fit_func(x_smooth, beta_bayes)
    y_fit_jeff = fit_func(x_smooth, beta_jeff)
    
    # Format beta values in scientific notation
    def format_beta(beta):
        exp = int(np.floor(np.log10(abs(beta))))
        coef = beta / 10**exp
        return f'${coef:.2f}\\times10^{{{exp}}}$'
    
    # Create the plot
    ax.scatter(x_data, bayes_data, color='purple', 
              label=f'Beta-BIQAE (β={format_beta(beta_bayes)}, R²={r2_bayes:.4f})', 
              s=40, marker='^')
    ax.plot(x_smooth, y_fit_bayes, '-', color='purple', alpha=0.5, linewidth=2)
    
    # Plot Jeff data points and fit line
    ax.scatter(x_data, jeff_data, color='green', 
              label=r'IQAE$_{CP}$'+ f' (β={format_beta(beta_jeff)}, R²={r2_jeff:.4f})', 
              s=40, facecolors='none', edgecolors='green')
    ax.plot(x_smooth, y_fit_jeff, '-', color='green', alpha=0.5, linewidth=2)
    
    # Calculate improvement percentages
    improvements = [(j - b)/j * 100 for b, j in zip(bayes_data, jeff_data)]
    
    # Calculate theoretical improvement from beta values
    beta_improvement = (beta_jeff - beta_bayes)/beta_jeff * 100
    
    # Create inset axes for improvement plot - increased width and height
    inset_height = 0.28 if epsilon == 0.01 else 0.32
    ax_inset = plt.axes([0.3, 0.15, 0.45, inset_height])  # [left, bottom, width, height]
    
    # Plot improvements in inset - only line, no points
    ax_inset.fill_between(x_data, improvements, color='orange', alpha=0.3)
    ax_inset.plot(x_data, improvements, color='orange', linewidth=1)
    
    # Add horizontal dotted line for improvement calculated from beta values
    ax_inset.axhline(y=beta_improvement, color='red', linestyle='--', linewidth=2, 
                     label=f'From β-values: {beta_improvement:.1f}%')
    ax_inset.legend(fontsize=10)
    
    # Configure inset axes with larger font sizes
    ax_inset.set_xticks(np.arange(0, 1.1, 0.2))
    ax_inset.tick_params(axis='both', labelsize=12)  # Increased from 10
    ax_inset.set_ylabel('Improvement (%)', fontsize=12, labelpad=2)  # Increased from 10
    ax_inset.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    
    # Main plot configuration
    ax.set_xlabel('Amplitude', fontsize=16)
    ax.set_ylabel('Quantum Sample Complexity', fontsize=16)
    ax.set_title(f'Complexity Comparison (ε = {epsilon})', fontsize=18)
    
    # Set y-axis limits with some padding
    max_y = max(max(bayes_data), max(jeff_data))
    ax.set_ylim(0, max_y * 1.2)  # Add 20% padding to the top
    
    ax.legend(fontsize=12, loc='upper left')
    
    # Set x-ticks from 0 to 1
    ax.set_xticks(np.arange(0, 1.1, 0.2))
    
    # Set tick parameters fontsize to 14
    ax.tick_params(axis='both', labelsize=14)
    
    # Save the figure with reduced DPI
    plt.savefig(f'all_angle_plot_epsilon_{epsilon}.png', dpi=300, bbox_inches='tight')
    plt.close()